In [1]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
import boto3
from nba_api.stats.endpoints import LeagueGameFinder, LeagueDashPlayerBioStats
import psycopg2


pd.set_option('display.max_columns', 500)

# Example usage
db_name = "thebigone"
username = "ajwin"
password = "CharlesBark!23"
host = "nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com"


In [ ]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = []
for i in list(range(1983,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]


In [3]:
# CREATE DATABASE
# import boto3

# # Create RDS Client
# rds_client = boto3.client('rds')

# def create_rds_instance(db_instance_identifier, db_name, master_username, master_user_password):
#     try:
#         response = rds_client.create_db_instance(
#             DBInstanceIdentifier=db_instance_identifier,
#             AllocatedStorage=20,  # Adjust as needed
#             DBName=db_name,
#             Engine='postgres',  # Using PostgreSQL, can switch if needed
#             MasterUsername=master_username,
#             MasterUserPassword=master_user_password,
#             DBInstanceClass='db.t3.micro',  # Adjust based on size
#             PubliclyAccessible=False,
#             MultiAZ=False,
#             StorageType='gp2',
#             BackupRetentionPeriod=7,
#         )
#         print(f"RDS instance {db_instance_identifier} creation initiated.")
#         return response
#     except Exception as e:
#         print(f"Error creating RDS instance: {str(e)}")

# create_rds_instance("nba-rds-instance","thebigone","ajwin","CharlesBark!23")


RDS instance nba-rds-instance creation initiated.


{'DBInstance': {'DBInstanceIdentifier': 'nba-rds-instance',
  'DBInstanceClass': 'db.t3.micro',
  'Engine': 'postgres',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'ajwin',
  'DBName': 'thebigone',
  'AllocatedStorage': 20,
  'PreferredBackupWindow': '06:40-07:10',
  'BackupRetentionPeriod': 7,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-048e804df880ce6ca',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'default',
   'DBSubnetGroupDescription': 'default',
   'VpcId': 'vpc-0da11b2b953a7cab5',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-02fedebb493b9a06e',
     'SubnetAvailabilityZone': {'Name': 'us-east-1c'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-0c8f0f9dc389265c1',
     'SubnetAvailabilityZone': {'Name': 'us-east-1e'},
     'S

In [9]:
seasons = []
for i in list(range(1983,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]


def get_games_data():
    # Get game data
    gamesl = []
    for i in seasons:
        gamefinder = LeagueGameFinder(league_id_nullable='00',season_nullable=i).get_data_frames()[0]
        gamesl.append(gamefinder)
        time.sleep(1)
    gamehistory = pd.concat(gamesl,axis=0)
    gamehistory['GAME_DATE'] =  pd.to_datetime(gamehistory['GAME_DATE'])
    return gamehistory


def get_players_data():
    # Get player data
    playersl = []
    for i in seasons:
        playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
        playerfinder['season']=i
        playersl.append(playerfinder)
        time.sleep(.5)
    allplayers = pd.concat(playersl,axis=0)
    return allplayers


In [10]:

# Connect to RDS database
def connect_to_rds(db_name, username, password, host, port=5432):
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=username,
            password=password,
            host=host,
            port=port
        )
        print("Connected to RDS database")
        return conn
    except Exception as e:
        print(f"Error connecting to RDS: {str(e)}")
        return None

# Create a table in PostgreSQL
def create_table(conn, table_name, dataframe):
    columns = ", ".join([f"{col} TEXT" for col in dataframe.columns])  # Simplified, adjust as needed
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns});"
    
    cursor = conn.cursor()
    cursor.execute(create_table_query)
    conn.commit()
    print(f"Table {table_name} created successfully.")

# Insert data into the table
def insert_data(conn, table_name, dataframe):
    cursor = conn.cursor()
    for _, row in dataframe.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        columns = ', '.join(dataframe.columns)
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"
        cursor.execute(insert_query, tuple(row))
    conn.commit()
    print(f"Data inserted into {table_name}.")


conn = connect_to_rds(db_name, username, password, host)

# Get and insert games data
games_data = get_games_data()
create_table(conn, "games", games_data)
insert_data(conn, "games", games_data)

# Get and insert players data
players_data = get_players_data()
create_table(conn, "players", players_data)
insert_data(conn, "players", players_data)


Connected to RDS database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_54512\3690715273.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gamehistory = pd.concat(gamesl,axis=0)


Table games created successfully.
Data inserted into games.


C:\Users\ajwin\AppData\Local\Temp\ipykernel_54512\3690715273.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  allplayers = pd.concat(playersl,axis=0)


Table players created successfully.
Data inserted into players.


Connected to RDS database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_54512\270539326.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


In [ ]:
#############################################################
#############################################################

# Query the database and return results as a pandas dataframe
def query_database_to_dataframe(conn, query):
    try:
        # Read the query result into a pandas dataframe
        dataframe = pd.read_sql(query, conn)
        return dataframe
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        return None

# Connect to the RDS database
conn = connect_to_rds(db_name, username, password, host)

# Define your SQL query
query = "SELECT * FROM players;"  # Example query: fetch 10 rows from the 'games' table

# Execute the query and store the result in a pandas dataframe
if conn:
    result_df = query_database_to_dataframe(conn, query)


# Close the connection
conn.close()